In [1]:
import tensorflow as tf
import keras 
from keras.applications import DenseNet201
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers

Using TensorFlow backend.


In [2]:
conv_base = DenseNet201(weights='imagenet', include_top = False, input_shape=(300,300,3))

74842112/74836368 [==============================] - 3s 0us/step


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

In [5]:
X = np.load("gdrive/My Drive/pcb/xtrain.npy")
y = np.load("gdrive/My Drive/pcb/ytrain.npy")

In [ ]:
conv_base.predict(X[0].reshape(1,300,300,3)).shape

(1, 9, 9, 1920)

In [ ]:
# Training set has 472 samples

Training with k-fold Validation

In [ ]:
X.shape, (y == 0).sum(), (y == 1).sum()

((472, 300, 300, 3), 236, 236)

In [6]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)


datagen.fit(X)

In [7]:
nos = X.shape[0]
bs = int(nos/8)


numit = NumpyArrayIterator(
    X, y, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

samples = np.zeros(shape=(nos,9,9,1920))
labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    samples[i*bs:(i+1)*bs] = f
    labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)


In [8]:
samples = samples.reshape(nos,9*9*1920)

In [ ]:
samples.shape, labels.shape

((472, 155520), (472,))

In [9]:
# To save memory
del X
del y

In [10]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)

accuracies = []
f = 1

for train_index, test_index in skf.split(samples, labels):
        X_train = samples[train_index]
        X_test = samples[test_index]
        y_train = labels[train_index]
        y_test = labels[test_index]
      
        model = models.Sequential()
        model.add(layers.Dense(256, activation = 'relu', input_dim = 9*9*1920))
        # model.add(layers.Dropout(0.5))
        model.add(layers.Dense(256, activation = 'relu'))
        model.add(layers.Dense(128, activation = 'relu'))
        model.add(layers.Dense(1, activation = 'sigmoid'))
        model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
        my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]

        print('Fold {}'.format(f))
        history = model.fit(X_train, y_train, epochs = 100, batch_size = 20, validation_data = (X_test, y_test), shuffle = True, callbacks = my_callbacks)

        test_preds = model.predict(X_test, batch_size = 20)
        test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

        accuracies.append([confusion_matrix(y_test,test_predclass)[0][0]/(y_test == 0.0).sum(), confusion_matrix(y_test,test_predclass)[1][1]/(y_test == 1.0).sum()])
        f+=1


Fold 1
Train on 413 samples, validate on 59 samples
Epoch 1/100
413/413 [==============================] - 1s 3ms/step - loss: 5.1104 - accuracy: 0.5811 - val_loss: 8.5515 - val_accuracy: 0.4915
Epoch 2/100
413/413 [==============================] - 1s 2ms/step - loss: 1.7816 - accuracy: 0.7724 - val_loss: 0.8479 - val_accuracy: 0.7288
Epoch 3/100
413/413 [==============================] - 1s 2ms/step - loss: 0.2550 - accuracy: 0.9274 - val_loss: 1.4937 - val_accuracy: 0.7458
Epoch 4/100
413/413 [==============================] - 1s 2ms/step - loss: 0.1036 - accuracy: 0.9588 - val_loss: 0.6023 - val_accuracy: 0.8475
Epoch 5/100
413/413 [==============================] - 1s 2ms/step - loss: 0.1227 - accuracy: 0.9782 - val_loss: 0.6279 - val_accuracy: 0.7966
Epoch 6/100
413/413 [==============================] - 1s 2ms/step - loss: 0.0259 - accuracy: 0.9903 - val_loss: 1.0525 - val_accuracy: 0.7458
Epoch 7/100
413/413 [==============================] - 1s 2ms/step - loss: 0.0294 - accura

In [11]:
zero_acc = 0
one_acc = 0

for i in accuracies:
  zero_acc+= i[0]
  one_acc+=i[1]

zero_acc/=8
one_acc/=8

print(zero_acc, one_acc)

0.8518678160919539 0.7958333333333334


Training on whole Training Dataset

In [12]:
model = models.Sequential()
model.add(layers.Dense(512, activation = 'relu', input_dim = 9*9*1920))
# model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))
model.compile(optimizer = optimizers.Adam(lr = 1e-3),loss = 'binary_crossentropy',metrics = ['accuracy'])
my_callbacks = [EarlyStopping(monitor = 'val_loss', patience=5, restore_best_weights=True)]


In [13]:
xtest = np.load('gdrive/My Drive/pcb/xtest.npy')
ytest = np.load('gdrive/My Drive/pcb/ytest.npy')


In [14]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

In [15]:
datagentest.fit(xtest)

In [16]:
testnos = ytest.shape[0]

numittest = NumpyArrayIterator(
    xtest, ytest, datagentest, batch_size=testnos, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


test_samples = np.zeros(shape=(testnos,9,9,1920))
test_labels = np.zeros(shape=(testnos))
for samples_batch,labels_batch in numittest:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    test_samples[::] = g
    test_labels[:] = labels_batch
    break

(552, 9, 9, 1920)


In [17]:
test_samples = test_samples.reshape(testnos,9*9*1920)

In [18]:
historytest = model.fit(samples, labels, epochs = 100, batch_size = 20, validation_data = (test_samples, test_labels), shuffle = True,  callbacks = my_callbacks)

Train on 472 samples, validate on 552 samples
Epoch 1/100
472/472 [==============================] - 2s 4ms/step - loss: 5.1656 - accuracy: 0.6398 - val_loss: 4.9667 - val_accuracy: 0.4022
Epoch 2/100
472/472 [==============================] - 2s 4ms/step - loss: 1.6029 - accuracy: 0.7754 - val_loss: 1.4279 - val_accuracy: 0.8931
Epoch 3/100
472/472 [==============================] - 2s 4ms/step - loss: 1.0214 - accuracy: 0.8517 - val_loss: 1.8200 - val_accuracy: 0.6594
Epoch 4/100
472/472 [==============================] - 2s 4ms/step - loss: 0.2130 - accuracy: 0.9258 - val_loss: 1.0503 - val_accuracy: 0.7663
Epoch 5/100
472/472 [==============================] - 2s 4ms/step - loss: 0.1791 - accuracy: 0.9619 - val_loss: 1.9064 - val_accuracy: 0.6594
Epoch 6/100
472/472 [==============================] - 2s 4ms/step - loss: 0.1265 - accuracy: 0.9703 - val_loss: 1.0348 - val_accuracy: 0.8478
Epoch 7/100
472/472 [==============================] - 2s 4ms/step - loss: 0.0784 - accuracy: 0.

In [19]:
test_preds = model.predict(test_samples, batch_size = 20)

In [20]:
test_predclass = np.array([1.0 if i>0.5 else 0.0 for i in test_preds])

In [21]:
confusion_matrix(test_labels,test_predclass)[0][0]/(ytest == 0.0).sum(), confusion_matrix(test_labels,test_predclass)[1][1]/(ytest == 1.0).sum()

(0.6440677966101694, 0.8823529411764706)

In [ ]:
# Saving model

# model_json = model.to_json()
# with open("gdrive/My Drive/models/VGG19_12.json", "w") as json_file:
#     json_file.write(model_json)

In [ ]:
# model.save_weights("gdrive/My Drive/models/VGG19_12.h5")
# print("Saved model to disk")

In [ ]:
# Saving and loading models
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/

TypeError: ignored